## 1. Problems encountered in the map

### I have downloaded a map of the New York City area and found some of the problems as follows.

#### First lets find inconsistent postcode.

In [58]:
from pymongo import MongoClient

import re

client = MongoClient("mongodb://localhost:27017")
db = client.osm

postcode_regex = r'^[0-9]{5}$'

Actual number of postcodes

In [59]:
db.nyc.count({"addr.postcode": {"$exists": 1}})

36992

Number of consistent postcodes.

In [60]:
db.nyc.count({"addr.postcode": {"$regex": postcode_regex}})

36964

Lets not look at all the inconsistent postcodes

In [61]:
inconsistent_postcodes = db.nyc.find({"addr.postcode": {"$not": re.compile(postcode_regex), "$exists": 1}})

for item in inconsistent_postcodes:
    print(item['addr']['postcode'])

12
NY 10703
NY 10533
NY 10533
NY 10533
NY 10533
NY 10533
NY 10533
NY 10533
NY 10533
NY 10533
08854-8006
08901-2867
08854-8062
08854-8020
08901-8520
08901-8529
08901-8536
08901-8519
08901-2832
08854-8009
08854-8000
08854-8063
08854-8019
08854-8031
08901-8542
08854-8040
08901-8502


There are three types of inconsistencies in the data. The first can be removed by dropping the "NY" and trimming the string for white spaces. And the second can be corrected by dropping the data after the "-". The finally one is invalid postcode.

Lets, correct these two issues.

In [62]:
inconsistent_postcodes = db.nyc.find({"addr.postcode": {"$not": re.compile(postcode_regex), "$exists": 1}})

for item in inconsistent_postcodes:
    postcode = item['addr']['postcode']
    if postcode[:2] == 'NY':
        cleaned_data = postcode[2:].strip()
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.postcode": cleaned_data}})
    elif '-' in postcode:
        cleaned_data = postcode.split('-')[0]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.postcode": cleaned_data}})

#### Now, lets check to see if there are any inconsistencies in the street address

In [63]:
inconsistent_streets = db.nyc.find({"addr.street": {"$exists": 1}}).limit(50)

for item in inconsistent_streets:
    print(item['addr']['street'])

Convent Lane
Monroe St
1st St
Park Ave
Madison St
Jackson St
Clinton St
2nd St
Newark St
4th St
4th St
3rd St
Washington St
Adams St
Garden St
Adams St
Hudson St
7th St
Bloomfield St
Clinton St
Park Ave
Madison St
East 240th Street
Webb Avenue
Bainbridge Avenue
Braddock Avenue
Main Street
White Plains Road
East 2nd Street
Amsterdam Avenue
Front Street
Grand Street
St. Nicholas Avenue
Old Mill Road
Broadway
Ellis Pky
Broadway
North End Avenue
Chambers Street
Speedwell Ave
East 52nd Street
West Beech Street
West 58th Street
56th Street
South Orange Avenue
Bedford Avenue
Washington Avenue
Sterling Place
Nostrand Avenue
Schenectady Avenue


Three abbreviation are clearly noticeable. The first is that "St" is used instead of "Street" and "Ave." is used in place of "Avenue". Finally, "St." need to be changed to "St."

Lets fix these problems

In [73]:
street_regex = "((St|Pky|Ave\.)$)|(^St.\s)"

inconsistent_streets = db.nyc.find({"addr.street": {"$regex": re.compile(street_regex)}})

for item in inconsistent_streets:
    print(item['addr']['street'])

Lets fix these two abbreviations.

In [72]:
inconsistent_streets = db.nyc.find({"addr.street": {"$regex": re.compile(street_regex)}})

for item in inconsistent_streets:
    street = item['addr']['street']
    if street[-2:] == "St":
        cleaned_data = street + "reet"
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[:3] == "St.":
        cleaned_data = "Street" + street[4:]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[-4:] == "Ave.":
        cleaned_data = street[:-1] + "nue"
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[-3:] == "Pky":
        cleaned_data = street[:-3] + "Parkway"
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})

While querying for the above abbreviations we have found another abbreviation, "E" is used in place of "East". Lets try and look if there are more direction based streets.

In [74]:
direction_street_regex = "^[NSEW]\s"

inconsistent_streets = db.nyc.find({"addr.street": {"$regex": re.compile(direction_street_regex)}})

for item in inconsistent_streets:
    print(item['addr']['street'])

Lets fix this directional the abbreviations.

In [75]:
inconsistent_streets = db.nyc.find({"addr.street": {"$regex": re.compile(direction_street_regex)}})

for item in inconsistent_streets:
    street = item['addr']['street']
    if street[0] == "N":
        cleaned_data = "North" + street[1:]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[0] == "S":
        cleaned_data = "South" + street[1:]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[0] == "E":
        cleaned_data = "East" + street[1:]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[0] == "W":
        cleaned_data = "West" + street[1:]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})

## 2. Data Overview